In [ ]:
#import necessary libraries
from pathlib import Path
import pandas as pd
import networkx as nx
import numpy as np
import geopandas as gpd
from matplotlib.colors import LinearSegmentedColormap
import scipy.stats as stats

# Converting Node Observations into Network Observations

In [ ]:
PATH_TO_DATA = Path("..") / "Data" / "SurveyData.xlsx"

# Load data from Excel files
nodes_data = pd.read_excel(PATH_TO_DATA, sheet_name="LCRW_Survey_Data", header=0)
# nodes_data = pd.read_excel(PATH_TO_DATA, sheet_name="Data_LCRW_Modified_2025_02_17", header=1) #To include more surveys that before were 'locaiton not accessible' 2025_02_17
confluence_nodes = pd.read_excel(PATH_TO_DATA, sheet_name="LCRW_Ghost_Nodes")
stretches = pd.read_excel(PATH_TO_DATA, sheet_name="LCRW_Edges").astype(str)  # Convert all values to strings

In [ ]:
# Create a new DataFrame with boolean values
nodes_data_bool = nodes_data.copy()  # Copy the original DataFrame to avoid modifying it

#Apply to each column
survey_date = [col for col in nodes_data.columns if "202" in col]

for col in survey_date:
    nodes_data_bool[col] = nodes_data[col].map({
        "Yes": True,
        "No": False
    })

# nodes_data_bool

In [ ]:
def calculate_network(survey_date, nodes_data_bool, confluence_nodes, stretches):
    # Ensure the None values are correctly registered
    nodes_data_bool = nodes_data_bool.where(nodes_data_bool.notnull(), None)

    # Create directed network
    G = nx.DiGraph()

    # Create nodes and add the attribute corresponding to data
    for _, data in nodes_data_bool.iterrows():
        node_ID = str(data["Point ID"])
        G.add_node(node_ID, pos=(data['Longitude'], data['Latitude']), node_activity=data[survey_date])

    # Add confluence nodes
    for _, row in confluence_nodes.iterrows():
        G.add_node(row['Node'], pos=(row['Longitude'], row['Latitude']), node_activity=None)  # Set a default value

    # Explicitly set activity for a necessary nodes (e.g., those that correspond to Little Calumet River)
    g9_node = "g9"  
    G.nodes[g9_node]["node_activity"] = True
    
    g0_node = "g0"  
    G.nodes[g0_node]["node_activity"] = True
    
    # Add network edges // river stretches
    for _, row in stretches.iterrows():
        if row["Node Start"] in G.nodes and row["Node End"] in G.nodes:
            G.add_edge(row["Node Start"], row["Node End"])

    # Definitions to calculate the value of each node
    def find_upstream(node_ID):
        if G.nodes[node_ID].get("node_activity", None) is not None:
            return G.nodes[node_ID]["node_activity"]
        
        is_upstream_wet = None
        for up_index, _ in G.in_edges(node_ID):
            res = find_upstream(up_index)
            if res is not None:
                is_upstream_wet = is_upstream_wet or res
        return is_upstream_wet
        
    def find_downstream(node_ID):
        if G.nodes[node_ID].get("node_activity", None) is not None:
            return G.nodes[node_ID]["node_activity"]
        
        is_downstream_wet = None
        for _, down_index in G.out_edges(node_ID):
            res = find_downstream(down_index)
            if res is not None:
                is_downstream_wet = is_downstream_wet or res
        return is_downstream_wet

    # First pass: Initial assignment of node activity
    for node_ID in G.nodes:
        upstream = find_upstream(node_ID)
        downstream = find_downstream(node_ID)
        if upstream == downstream:
            G.nodes[node_ID]["node_activity"] = upstream
            
    # # Check that all nodes have been assigned
    # for node_ID in G.nodes:
    #     node_attrs = G.nodes[node_ID]
    #     if node_attrs.get("node_activity", None) is None:
    #         print(node_ID)

    # Second pass: Assign "disconnected" or "indeterminate"
    for node_ID in G.nodes:
        if G.nodes[node_ID]["node_activity"] is None:
            upstream = find_upstream(node_ID)
            downstream = find_downstream(node_ID)

            if upstream is not None and downstream is not None:
                if upstream != downstream:
                    G.nodes[node_ID]["node_activity"] = "disconnected"
            else:
                G.nodes[node_ID]["node_activity"] = None


    # Assign stretch attribute based on node activity
    for up_index, down_index in G.edges:
        up_activity = G.nodes[up_index].get("node_activity")
        down_activity = G.nodes[down_index].get("node_activity")
        
        if up_activity is True and down_activity is True:
            G.edges[up_index, down_index]["color"] = 'blue'
        elif up_activity is False and down_activity is False:
            G.edges[up_index, down_index]["color"] = 'red'
        elif up_activity is True and down_activity is False:
            G.edges[up_index, down_index]["color"] = 'orange'
        elif up_activity is False and down_activity is True:
            G.edges[up_index, down_index]["color"] = 'orange'
        elif "disconnected" in (up_activity, down_activity):
            G.edges[up_index, down_index]["color"] = 'orange'
        else:
            G.edges[up_index, down_index]["color"] = 'grey'

    # Assign activity to stretches
    for up_index, down_index in G.edges:
        up_activity = G.nodes[up_index].get("node_activity")
        down_activity = G.nodes[down_index].get("node_activity")
        
        if up_activity is True and down_activity is True:
            G.edges[up_index, down_index]["stretch_activity"] = 'Active'
        elif up_activity is False and down_activity is False:
            G.edges[up_index, down_index]["stretch_activity"] = 'Not Active'
        elif up_activity is True and down_activity is False:
            G.edges[up_index, down_index]["stretch_activity"] = 'Disconnected'
        elif up_activity is False and down_activity is True:
            G.edges[up_index, down_index]["stretch_activity"] = 'Disconnected'
        elif "disconnected" in (up_activity, down_activity):
            G.edges[up_index, down_index]["stretch_activity"] = 'Disconnected'
        else:
            G.edges[up_index, down_index]["stretch_activity"] = 'Not enough information'

    return G


In [ ]:
# Define a function to collect stretch activities into a DataFrame
def collect_stretch_activities(survey_dates, nodes_data_bool, confluence_nodes, stretches):
    # Dictionary to hold data for DataFrame creation
    edge_data_dict = {}

    for date in survey_dates:
        G = calculate_network(date, nodes_data_bool, confluence_nodes, stretches)
        
        # Collect data for the current survey date
        for edge in G.edges:
            stretch_status = G.edges[edge].get("stretch_activity", 'Not available')
            # Create a unique key for each edge
            edge_key = (edge[0], edge[1])
            if edge_key not in edge_data_dict:
                edge_data_dict[edge_key] = {}
            edge_data_dict[edge_key][f'{date}'] = stretch_status

    # Convert the dictionary into a DataFrame
    # Prepare a list of dictionaries to create the DataFrame
    all_stretch_data = []
    for (node_start, node_end), activities in edge_data_dict.items():
        # Merge the activities into a single dictionary with Node Start and Node End
        edge_info = {'Node Start': node_start, 'Node End': node_end}
        edge_info.update(activities)
        all_stretch_data.append(edge_info)

    # Create DataFrame from the list of dictionaries
    stretches_activity_df = pd.DataFrame(all_stretch_data)
    
   # Merge the stretch lengths into the DataFrame
    stretches_data = pd.merge(stretches_activity_df, stretches[['Node Start', 'Node End', 'Stretch Length (km)']], 
                                        on=['Node Start', 'Node End'], how='left')
    
    # Reorder columns to have "length" after "Node End"
    column_order = ['Node Start', 'Node End', 'Stretch Length (km)'] + [col for col in stretches_data.columns if col not in ['Node Start', 'Node End', 'Stretch Length (km)']]
    stretches_data = stretches_data[column_order]
    
    return stretches_data

# Collect stretch activities and create DataFrame
stretches_data = collect_stretch_activities(survey_date, nodes_data_bool, confluence_nodes, stretches)

#Print the full DataFrame // Set display options t5o see full df
pd.set_option('display.max_rows', None)  # None means unlimited rows
pd.set_option('display.max_columns', None)  # None means unlimited columns


In [ ]:
# Save the data as a pickle file
stretches_data.to_csv("LCRW_stretches_data.csv")

# Computing Persistency Index (P_i)

In [ ]:
#Calculate persistency index (P_i) for each stretch
def persistence_index(df):
    # Identify columns related to Stretch Activity
    activity_columns = [col for col in df.columns if '202' in col]
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_numeric = df.copy()
    
    # Replace activity status with numeric values
    pd.set_option('future.no_silent_downcasting', True) #To avoid future warning
    
    df['Stretch Length (km)'] = pd.to_numeric(df['Stretch Length (km)'], errors='coerce')
    df_numeric[activity_columns] = df_numeric[activity_columns].replace({
        'Active': 1,
        'Not Active': 0,
        'Disconnected':0.5,
        'Not enough information': np.nan
    }).astype(float)
    
    # Calculate the number of 'Active' states
    df['Active Count'] = df_numeric[activity_columns].sum(axis=1)
    
    # Calculate the total number of valid observations
    df['Valid Observations Count'] = df_numeric[activity_columns].notna().sum(axis=1)
    
    # Compute the proportion of 'Active' states
    df['Persistency Index (P_i)'] = df['Active Count'] / df['Valid Observations Count']
    
    # Drop intermediate columns used for calculation
    df.drop(columns=['Active Count', 'Valid Observations Count'], inplace=True)
    
    # Reorder columns to place Persistency Index next to Stretch Length column
    columns = list(df.columns)
    stretch_length_index = columns.index('Stretch Length (km)')
    # Move 'Persistency Index (P_i)' next to 'Stretch Length (km)'
    columns.insert(stretch_length_index + 1, columns.pop(columns.index('Persistency Index (P_i)')))
    df = df[columns]
    

    return df

# Add the activity proportion column to the existing DataFrame
stretches_data = persistence_index(stretches_data)

# #Print the full DataFrame // Set display options t5o see full df
# pd.set_option('display.max_rows', None)  # None means unlimited rows
# pd.set_option('display.max_columns', None)  # None means unlimited columns

# stretches_data

In [ ]:
# Count how many stretches have Persistency Index (P_i) = 1
persistency_1_count = stretches_data[stretches_data['Persistency Index (P_i)'] == 1].shape[0]
print(f"Number of stretches with persistency = 1: {persistency_1_count}")

# Calculate the total length of stretches with Persistency Index (P_i) = 1
total_length_persistency_1 = stretches_data[stretches_data['Persistency Index (P_i)'] == 1]['Stretch Length (km)'].sum()
print(f"Total length of stretches with persistency = 1: {total_length_persistency_1:.2f} km")

# Total length of all stretches
total_length = stretches_data['Stretch Length (km)'].sum()
print(f"Total length: {total_length:.2f} km")

# Calculate ephemeral percentage
percentage_ephemeral = ((total_length - total_length_persistency_1) / total_length) * 100
print(f"Ephemeral percentage is: {percentage_ephemeral:.2f}%")

# Mean Persistency Index
mean_pi = stretches_data['Persistency Index (P_i)'].mean()
print(f"Mean Persistency Index (P_i): {mean_pi:.3f}")

# Mean P_i where P_i ≠ 1
mean_pi_not_1 = stretches_data[stretches_data['Persistency Index (P_i)'] != 1]['Persistency Index (P_i)'].mean()
print(f"Mean Persistency Index (P_i) for stretches where P_i ≠ 1: {mean_pi_not_1:.3f}")

In [ ]:
#Import all geospatial data
catchment_outline_gdf = gpd.read_file(Path("..") / "Data" / "CatchmentBoundary" / "WestLittleCal_Delineation.shp")

tributaries = gpd.read_file(Path("..") / "Data" / "TributaryBoundaries" / "tributaries.shp")
tributaries = tributaries.to_crs(epsg=3857)
tributaries["ID"] =["LCRW", "IBP", "CalUnion", "CherryCreek"]

land_use = gpd.read_file(Path("..") / "Data" / "LandUse" / "Catchment_LandUse.shp")

land_cover = Path("..") / "Data" / "LandCover" / "Catchment_LandCover.tif"

river_network = gpd.read_file(Path("..") / "Data" / "RiverNetwork" / "Catchment_RiverNetwork.shp")
# Filter for the specific FID
river_255 = river_network.loc[[255]]  # Double brackets to keep it a GeoDataFrame
# Ensure it's in the correct CRS
river_255 = river_255.to_crs(epsg=3857)
# Clip river network to the extent of the land use data
river_fid_255 = gpd.clip(river_255, land_use)


In [ ]:
def reproject_raster_to_3857(raster_path):
    import rasterio
    from rasterio.warp import calculate_default_transform, reproject, Resampling

    with rasterio.open(raster_path) as src:
        dst_crs = 'EPSG:3857'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        destination = np.empty((src.count, height, width), dtype=src.dtypes[0])

        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=destination[i - 1],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

        return destination, transform, kwargs

In [ ]:
def plot_binary_pi_map(
    nodes_data_bool, confluence_nodes, stretches,
    catchment_outline_gdf, river_network, tributaries,
    land_cover, land_use
):
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    from matplotlib.collections import LineCollection
    import matplotlib.patheffects as pe
    import geopandas as gpd
    import networkx as nx
    from shapely.geometry import LineString
    import numpy as np

    # Create directed graph
    G = nx.DiGraph()

    # Add nodes
    for _, data in nodes_data_bool.iterrows():
        node_ID = str(data["Point ID"])
        G.add_node(node_ID, pos=(data['Longitude'], data['Latitude']))
    for _, row in confluence_nodes.iterrows():
        G.add_node(row['Node'], pos=(row['Longitude'], row['Latitude']))

    # Add edges with P_i
    for _, row in stretches.iterrows():
        if row["Node Start"] in G.nodes and row["Node End"] in G.nodes:
            G.add_edge(row["Node Start"], row["Node End"], P_i=row["Persistency Index (P_i)"])

    
    # Positions and attributes
    positions = nx.get_node_attributes(G, "pos")
    edges = list(G.edges())
    pi_values = nx.get_edge_attributes(G, "P_i")

    # Binary color map: orange if P_i < 1, blue if P_i == 1
    edge_colors = ["#fff828ff" if pi_values.get(edge, 0) < 1 else '#0085b2' for edge in edges]
    edge_lines = [LineString([positions[u], positions[v]]) for u, v in edges]
    edge_gdf = gpd.GeoDataFrame(geometry=edge_lines, crs="EPSG:4326")

    # Reproject data
    edge_gdf = edge_gdf.to_crs(epsg=3857)
    catchment_outline_gdf = catchment_outline_gdf.to_crs(epsg=3857)
    river_network = river_network.to_crs(epsg=3857)
    tributaries = tributaries.to_crs(epsg=3857)
    land_use = land_use.to_crs(epsg=3857)

    # Clip and prepare background
    land_cover_data, land_cover_transform, _ = reproject_raster_to_3857(land_cover)
    custom_colors = ["#efefefee", "#c9c9c9", "#9e9d9d"]
    custom_cmap = LinearSegmentedColormap.from_list("impervious_cmap", custom_colors, N=256)
    extent = [
        land_cover_transform[2],
        land_cover_transform[2] + land_cover_transform[0] * land_cover_data.shape[2],
        land_cover_transform[5] + land_cover_transform[4] * land_cover_data.shape[1],
        land_cover_transform[5]
    ]

    fig, ax = plt.subplots(figsize=(10, 8))

    ax.imshow(
        land_cover_data[0],
        cmap=custom_cmap,
        extent=extent,
        interpolation='none',
        origin='upper',
        zorder=0
    )

    # Zoom margins
    pixel_width = abs(land_cover_transform.a)
    pixel_height = abs(land_cover_transform.e)
    ax.set_xlim(extent[0] + 40 * pixel_width, extent[1] - 40 * pixel_width)
    ax.set_ylim(extent[2] + 10 * pixel_height, extent[3] - 10 * pixel_height)

    # Plot data layers
    river_network_outside = gpd.overlay(river_network, catchment_outline_gdf, how='difference')
    river_network_outside.plot(ax=ax, color='#1c3652', linewidth=1, zorder=1)
    river_fid_255.plot(ax=ax, color='#1c3652', linewidth=1)
    tributaries.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, linestyle='--', zorder=4)
    catchment_outline_gdf.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, zorder=3)

    edge_collection = LineCollection(
        [list(line.coords) for line in edge_gdf.geometry],
        colors=edge_colors,
        linewidths=2.5,
        path_effects=[pe.withStroke(linewidth=5, foreground="#000000")],
        zorder=5
    )
    ax.add_collection(edge_collection)

    # Colorbar for interpretation
    import matplotlib.patches as mpatches
    legend_elements = [
        mpatches.Patch(color="#0085b2", label='P_i = 1 (Persistent)'),
        mpatches.Patch(color="#f3eb0f", label='P_i < 1 (Ephemeral)')
    ]
    # ax.legend(handles=legend_elements, loc='lower right', frameon=True)

    # Imperviousness colorbar
    cbar_raster = plt.colorbar(
        plt.cm.ScalarMappable(cmap=custom_cmap, norm=mcolors.Normalize(vmin=land_cover_data.min(), vmax=land_cover_data.max())),
        ax=ax,
        orientation='horizontal',
        fraction=0.03,
        pad=0.05,
        shrink=0.7
    )
    cbar_raster.set_label('Percent Impervious Surface (%)')

    ax.set_axis_off()
    plt.savefig("Ephemeral_Classification.png", bbox_inches='tight', dpi=300)
    plt.show()

    return G


In [ ]:
plot_binary_pi_map(nodes_data_bool, confluence_nodes, stretches_data, catchment_outline_gdf, river_network, tributaries, land_cover, land_use)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

# Trim off the brightest part of 'GnBu'
base_cmap = plt.get_cmap('GnBu')  # This works in all versions
cmap = LinearSegmentedColormap.from_list(
    'trimmed_GnBu',
    base_cmap(np.linspace(0, 0.8, 256))  # Avoid dark tones
)

def calculate_network_with_pi(
    nodes_data_bool, confluence_nodes, stretches,
    catchment_outline_gdf, river_network, tributaries,
    land_cover, land_use
):
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    from matplotlib.collections import LineCollection
    import matplotlib.patheffects as pe
    import geopandas as gpd
    import networkx as nx
    from shapely.geometry import LineString
    from matplotlib.colors import LinearSegmentedColormap
    import numpy as np

    # Create directed network graph
    G = nx.DiGraph()

    # Add nodes
    for _, data in nodes_data_bool.iterrows():
        node_ID = str(data["Point ID"])
        G.add_node(node_ID, pos=(data['Longitude'], data['Latitude']))
    for _, row in confluence_nodes.iterrows():
        G.add_node(row['Node'], pos=(row['Longitude'], row['Latitude']))

    # Add edges with Persistency Index
    for _, row in stretches.iterrows():
        if row["Node Start"] in G.nodes and row["Node End"] in G.nodes:
            G.add_edge(row["Node Start"], row["Node End"], Persistency_Index=row["Persistency Index (P_i)"])

    # Extract positions and edge persistency
    positions = nx.get_node_attributes(G, "pos")
    edges = list(G.edges())
    edge_persistency_index = nx.get_edge_attributes(G, "Persistency_Index")
    norm = mcolors.Normalize(vmin=0, vmax=1)
    # cmap = plt.get_cmap('GnBu')
    edge_colors = [cmap(norm(edge_persistency_index.get(edge, 0))) for edge in edges]
    edge_lines = [LineString([positions[u], positions[v]]) for u, v in edges]

    edge_gdf = gpd.GeoDataFrame(geometry=edge_lines, crs="EPSG:4326")
    node_gdf = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy(
            [pos[0] for pos in positions.values()],
            [pos[1] for pos in positions.values()]
        ),
        crs="EPSG:4326"
    )

    # Reproject all to EPSG:3857
    edge_gdf = edge_gdf.to_crs(epsg=3857)
    node_gdf = node_gdf.to_crs(epsg=3857)
    catchment_outline_gdf = catchment_outline_gdf.to_crs(epsg=3857)
    river_network = river_network.to_crs(epsg=3857)
    tributaries = tributaries.to_crs(epsg=3857)
    land_use = land_use.to_crs(epsg=3857)

    # Clip river network to land use extent
    river_network_clipped = gpd.clip(river_network, land_use)

    # Remove river parts inside catchment
    river_network_outside = gpd.overlay(river_network_clipped, catchment_outline_gdf, how='difference')
    

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Plot raster background
    land_cover_data, land_cover_transform, land_cover_meta = reproject_raster_to_3857(land_cover)

    # custom_colors = ['#6e898a', '#B0B0B0']  # dark greenish -> light gray
    custom_colors = ["#efefefee", "#c9c9c9", "#9e9d9d"]
    custom_cmap = LinearSegmentedColormap.from_list("impervious_cmap", custom_colors, N=256)

    extent = [
        land_cover_transform[2],  # xmin
        land_cover_transform[2] + land_cover_transform[0] * land_cover_data.shape[2],  # xmax
        land_cover_transform[5] + land_cover_transform[4] * land_cover_data.shape[1],  # ymin
        land_cover_transform[5]  # ymax
    ]

    ax.imshow(
        land_cover_data[0],  # single band raster
        cmap=custom_cmap,
        extent=extent,
        interpolation='none',
        origin='upper',
        zorder=0
    )

    # --- Zoom in by 10 pixels on all sides ---

    # Calculate pixel size from affine transform
    pixel_width = abs(land_cover_transform.a)
    pixel_height = abs(land_cover_transform.e)

    margin_x = 40 * pixel_width
    margin_y = 10 * pixel_height

    # Shrink extent by moving edges inward by 10 pixels worth of distance
    ax.set_xlim(extent[0] + margin_x, extent[1] - margin_x)
    ax.set_ylim(extent[2] + margin_y, extent[3] - margin_y)

    # -----------------------------------------

    # Set plot limits to land use bounds (clipping to land use extent)
    bounds = land_use.total_bounds  # xmin, ymin, xmax, ymax
    margin = 100  # meters, for some padding

    # Plot river network outside catchment
    river_network_outside.plot(ax=ax, color='#1c3652', linewidth=1, zorder=1)
    river_fid_255.plot(ax=ax, color='#1c3652', linewidth=1)

    # Plot edges with black outline
    edge_collection = LineCollection(
        [list(line.coords) for line in edge_gdf.geometry],
        colors=edge_colors,
        linewidths=2.5,
        path_effects=[pe.withStroke(linewidth=5, foreground="#000000")],
        zorder=5
    )
    ax.add_collection(edge_collection)

    # Plot tributaries
    tributaries.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, linestyle='--', zorder=4)

    # Plot catchment outline
    catchment_outline_gdf.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, zorder=3)

    # Persistency index colorbar - vertical right side
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cbar_pi = plt.colorbar(sm, ax=ax, orientation='vertical', fraction=0.03, pad=0.01, shrink=0.7)
    cbar_pi.set_label('Persistency Index $(P_i)$', rotation=90, labelpad=15)

    # Raster colorbar below plot (horizontal)
    cbar_raster = plt.colorbar(
        plt.cm.ScalarMappable(cmap=custom_cmap, norm=mcolors.Normalize(vmin=land_cover_data.min(), vmax=land_cover_data.max())),
        ax=ax,
        orientation='horizontal',
        fraction=0.03,  # smaller length
        pad=0.05,
        shrink=0.7       # smaller height
    )
    cbar_raster.set_label('Percent Impervious Surface (%)')

    # ax.set_title('Network Visualization of Persistency Index with Basemap')
    ax.set_axis_off()

    plt.savefig('Persistency_Index.png', bbox_inches='tight', dpi=300)
    plt.show()

    return G


In [ ]:
calculate_network_with_pi(nodes_data_bool, confluence_nodes, stretches_data, catchment_outline_gdf, river_network, tributaries, land_cover, land_use)

### Viewing impact of engineering on stretch activity

In [ ]:
stretches_type = pd.read_csv(Path("..") / "Data" / "StretchesData.csv")
stretches_type

# Prepare data for ANOVA
groups = []
for cat in ['Natural', 'Lined']:
    values = stretches_type.loc[stretches_type['Channel Type'] == cat, 'Persistency Index (P_i)']
    groups.append(values.dropna())

# One-way ANOVA
f_stat, p_val = stats.f_oneway(*groups)
print(f"ANOVA F-statistic: {f_stat:.3f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("There is a statistically significant difference between group means.")
else:
    print("No statistically significant difference between group means.")
    

# Computing ADNL

In [ ]:
def calculate_ADNL(df, survey_date):
    # Create dictionaries to store results for each date
    ADNL = {}
    ADNL_p = {}
    ADNL_p_o = {}
    total_length = {date: 0 for date in survey_date}

    # Process each survey date
    for date in survey_date:

        # Create a directed graph for the current date
        G = nx.DiGraph()

        # Add edges to the graph for the current date
        for _, row in df.iterrows():
            G.add_edge(row['Node Start'], row['Node End'], 
                       length=row['Stretch Length (km)'], 
                       activity=row[date])

        # Find all downstream nodes (nodes with no outgoing edges)
        downstream_nodes = [node for node in G.nodes if G.out_degree(node) == 0]

        def dfs(node, visited):
            if node in visited:
                return  # Exit if this node has already been processed
            visited.add(node)  # Mark the node as visited

            incoming_edges = list(G.in_edges(node))
            for u, v in incoming_edges:
                activity = G.edges[u, v].get('activity', '')
                stretch_length = G.edges[u, v].get('length', 0)

                if activity == 'Active':
                    total_length[date] += stretch_length
                    dfs(u, visited)

        # Start DFS from each downstream node
        for node in downstream_nodes:
            visited = set()  # Reset visited for each downstream node
            dfs(node, visited)

        # Calculate the total active length for the current survey date
        ADNL[date] = total_length[date]

        # Calculate total stretch length from the stretches_data DataFrame
        total_stretch_length = stretches_data['Stretch Length (km)'].sum()

        # Filter rows where the activity status is 'Active', 'Inactive', or 'Disconnected'
        valid_rows = df[date].isin(['Active', 'Not Active', 'Disconnected'])
        total_observed_stretch_length = df.loc[valid_rows, 'Stretch Length (km)'].sum()

        # Calculate the percentage of ADNL based on the total stretch length
        ADNL_p[date] = ADNL[date] / total_stretch_length if total_stretch_length > 0 else np.nan

        # Calculate the percentage of ADNL based on the observed stretch length
        ADNL_p_o[date] = ADNL[date] / total_observed_stretch_length if total_observed_stretch_length > 0 else np.nan

    # Convert results to a DataFrame
    ANDL_df = pd.DataFrame({
        'Survey Date': list(ADNL.keys()),
        'Active Network Drainage Length': list(ADNL.values()),
        'Percentage of Total Network': [ADNL_p[date] for date in ADNL.keys()],
        'Percentage of Observed Network': [ADNL_p_o[date] for date in ADNL.keys()]
    })

    return ANDL_df


In [ ]:
# Step 1: Calculate the ADNL DataFrame for all survey dates
ADNL_all = calculate_ADNL(stretches_data, survey_date)
ADNL_all

# Filter out rows where the percentage of observed network is not equal to the percentage of total network
ADNL_df = ADNL_all[ADNL_all['Percentage of Observed Network'] == ADNL_all['Percentage of Total Network']].copy()
ADNL_df

In [ ]:
# Group by 'Drainage_ID' and calculate the max and min total length for each group
drainage_length_stats = ADNL_df['Percentage of Total Network'].agg(['max', 'min', 'mean', 'median', 'std',])
drainage_length_stats

In [ ]:
ADNL_all.to_pickle('LCRW_ActiveNetworkDrainageLength.pkl')

In [ ]:
# Filter rows where the observed percentage matches the total percentage; no missing information
ADNL_LCRW = ADNL_all[ADNL_all['Percentage of Observed Network'] == ADNL_all['Percentage of Total Network']]

ADNL_LCRW_copy = ADNL_LCRW.copy()
ADNL_LCRW

In [ ]:
# Extract the corresponding survey dates for valid data points
survey_dates = ADNL_LCRW['Survey Date']
survey_dates = pd.to_datetime(ADNL_LCRW['Survey Date'], format='%Y_%m_%d')
survey_dates = survey_dates.dt.tz_localize('UTC')
ADNL_LCRW = ADNL_LCRW.set_index('Survey Date')
ADNL_LCRW.index.name = None
ADNL_LCRW

## Temporal Analysis: Relationship to Antecedent Conditions

### Antecedent Precipitation & Dry Days

In [ ]:
gridded_precip_LCRW = pd.read_parquet(Path("..") / "Data" / "Precipitation_24h_Jan24toJun25(UTC-6).parquet")

# To ensure it's a DataFrame, reset the index
gridded_precip_LCRW_df = gridded_precip_LCRW.reset_index()

# Rename the column 'timestamp' to 'datetime'
gridded_precip_LCRW_df.rename(columns={'timestamp': 'datetime'}, inplace=True)

# gridded_precip_LCRW_df

In [ ]:
# Use a copy of the original precip data
OG_precip = gridded_precip_LCRW.copy()

# Make sure the index is datetime and sorted
OG_precip.index = pd.to_datetime(OG_precip.index)
OG_precip.sort_index(inplace=True)

# Initialize the dry day counter for the LCRW column
dry_days = []
counter = 0

# Count consecutive dry days BEFORE each date
for precip in OG_precip['LCRW']:
    dry_days.append(counter)
    # if precip == 0:
    if precip < 9.99:
        counter += 1
    else:
        counter = 0

# Add result as new column
OG_precip['Dry_Days_Before_LCRW'] = dry_days

# Strip time component from the OG_precip index for merging
OG_precip = OG_precip.copy()
OG_precip['Date'] = OG_precip.index.date  # datetime.date objects

# Keep only relevant column(s)
dry_days_df = OG_precip[['Date', 'Dry_Days_Before_LCRW']].drop_duplicates()
dry_days_df = dry_days_df.set_index('Date')
# dry_days_df

In [ ]:
def calculate_antecedent_precip(survey_dates, site_data, max_days=60):  # Note site_data is the df with the effective precipitation data for each day
    
    # Make a copy of site_data to avoid modifying the original DataFrame
    site_data_copy = site_data.copy()

    # Initialize an empty list to hold the results for the single site
    results = {f'{survey_date}': [] for survey_date in survey_dates}

    # Ensure 'datetime' is in the correct format and set it as the index in the copied DataFrame
    site_data_copy['datetime'] = pd.to_datetime(site_data_copy['datetime'])  # Ensure the 'datetime' column is datetime
    site_data_copy.set_index('datetime', inplace=True)  # Set 'datetime' as the index for easy comparison

    # Loop over each survey date
    for survey_date in survey_dates:
        # Ensure survey_date is a datetime object
        survey_date = pd.to_datetime(survey_date) if isinstance(survey_date, str) else survey_date

        # Calculate antecedent effective precipitation for each window (1 to max_days)
        for days in range(1, max_days + 1):
           
            start_date = survey_date - pd.Timedelta(days=days)
           
            # Filter data for the current site within the date window
            site_data_in_window = site_data_copy.loc[(site_data_copy.index >= start_date) & (site_data_copy.index <= survey_date)]
            
            # Calculate the antecedent precipitation by summing the precipitation values in the window
            antecedent_precip = site_data_in_window['LCRW'].sum() if not site_data_in_window.empty else None
            
            # Store the result in the dictionary with the key as the survey date
            results[f'{survey_date}'].append(antecedent_precip)

    # Convert the results dictionary into a DataFrame
    result_df = pd.DataFrame(results)

    # Set the index to be the time windows (1 to 60 days)
    result_df.index = [f'{days}_days' for days in range(1, max_days + 1)]

    return result_df

Antecedent_Precip_LCRW = calculate_antecedent_precip(survey_dates, gridded_precip_LCRW_df)
# Antecedent_Precip_LCRW.head()

### Antecedent Evapotranspiration

In [ ]:
# Using API query in get_openet_polygon(simple).ipynb
LCRW_ET_data = pd.read_parquet(Path("..") / "Data" / "ET_24h_Jun'25.parquet")

LCRW_ET_data.reset_index(inplace=True)
# Rename the column 'timestamp' to 'datetime'
LCRW_ET_data.rename(columns={'time': 'datetime'}, inplace=True)

# Convert ET from in to mm
LCRW_ET_data["et_mm"] = LCRW_ET_data["et"]*25.4
LCRW_ET_data["datetime"] = LCRW_ET_data["datetime"].dt.date
# LCRW_ET_data

LCRW_ET_data['datetime'] = pd.to_datetime(LCRW_ET_data['datetime'])

# Localize to the local timezone (e.g., 'America/New_York') and then convert to UTC
LCRW_ET_data['datetime'] = LCRW_ET_data['datetime'].dt.tz_localize('America/New_York').dt.tz_convert('UTC')

In [ ]:
def calculate_antecedent_ET(survey_dates, site_data, max_days=60):  # Note site_data is the df with the effective precipitation data for each day
    
    # Make a copy of site_data to avoid modifying the original DataFrame
    site_data_copy = site_data.copy()

    # Initialize an empty list to hold the results for the single site
    results = {f'{survey_date}': [] for survey_date in survey_dates}

    # Ensure 'datetime' is in the correct format and set it as the index in the copied DataFrame
    site_data_copy['datetime'] = pd.to_datetime(site_data_copy['datetime'])  # Ensure the 'datetime' column is datetime
    site_data_copy.set_index('datetime', inplace=True)  # Set 'datetime' as the index for easy comparison

    # Loop over each survey date
    for survey_date in survey_dates:
        # Ensure survey_date is a datetime object
        survey_date = pd.to_datetime(survey_date) if isinstance(survey_date, str) else survey_date

        # Calculate antecedent effective precipitation for each window (1 to max_days)
        for days in range(1, max_days + 1):
           
            start_date = survey_date - pd.Timedelta(days=days)
           
            # Filter data for the current site within the date window
            site_data_in_window = site_data_copy.loc[(site_data_copy.index >= start_date) & (site_data_copy.index <= survey_date)]
            
            # Calculate the antecedent precipitation by summing the precipitation values in the window
            antecedent_precip = site_data_in_window['et_mm'].sum() if not site_data_in_window.empty else None
            
            # Store the result in the dictionary with the key as the survey date
            results[f'{survey_date}'].append(antecedent_precip)

    # Convert the results dictionary into a DataFrame
    result_df = pd.DataFrame(results)

    # Set the index to be the time windows (1 to 60 days)
    result_df.index = [f'{days}_days' for days in range(1, max_days + 1)]

    return result_df

In [ ]:
Antecedent_ET_LCRW = calculate_antecedent_ET(survey_dates, LCRW_ET_data)

### Antecedent Effective Precipitation

In [ ]:
# Normalize both datetime columns to only the date part
gridded_precip_LCRW_df['date'] = gridded_precip_LCRW_df['datetime'].dt.date
LCRW_ET_data['date'] = LCRW_ET_data['datetime'].dt.date

# Merge the result with thiessen_precip_data on 'datetime'
LCRW_EP_data = pd.merge(LCRW_ET_data[['date', 'et_mm']], 
                   gridded_precip_LCRW_df[['date', 'LCRW']], 
                   on='date', 
                   how='left')


# Subtract the ET from Precip NOT Cumulative
LCRW_EP_data["Effective Precipitation"] = np.where(
    LCRW_EP_data["LCRW"] >= LCRW_EP_data["et_mm"], 
    LCRW_EP_data["LCRW"] - LCRW_EP_data["et_mm"], 
    LCRW_EP_data["LCRW"]
)

# Display the merged DataFrame
# LCRW_EP_data

In [ ]:
# Convert the 'date' column to datetime (time will be set to 00:00:00)
LCRW_EP_data['datetime'] = pd.to_datetime(LCRW_EP_data['date'])

# Localize the datetime column to UTC
LCRW_EP_data['datetime'] = LCRW_EP_data['datetime'].dt.tz_localize('UTC')

# Verify the dtype is datetime64
# print(LCRW_EP_data['datetime'].dtype)
# LCRW_EP_data

In [ ]:
def calculate_antecedent_EP(survey_dates, site_data, max_days=60):  # Note site_data is the df with the effective precipitation data for each day
    
    # Make a copy of site_data to avoid modifying the original DataFrame
    site_data_copy = site_data.copy()

    # Initialize an empty list to hold the results for the single site
    results = {f'{survey_date}': [] for survey_date in survey_dates}

    # Ensure 'datetime' is in the correct format and set it as the index in the copied DataFrame
    site_data_copy['datetime'] = pd.to_datetime(site_data_copy['datetime'])  # Ensure the 'datetime' column is datetime
    site_data_copy.set_index('datetime', inplace=True)  # Set 'datetime' as the index for easy comparison

    # Loop over each survey date
    for survey_date in survey_dates:
        # Ensure survey_date is a datetime object
        survey_date = pd.to_datetime(survey_date) if isinstance(survey_date, str) else survey_date

        # Calculate antecedent effective precipitation for each window (1 to max_days)
        for days in range(1, max_days + 1):
           
            start_date = survey_date - pd.Timedelta(days=days)
           
            # Filter data for the current site within the date window
            site_data_in_window = site_data_copy.loc[(site_data_copy.index >= start_date) & (site_data_copy.index <= survey_date)]
            
            # Calculate the antecedent precipitation by summing the precipitation values in the window
            antecedent_precip = site_data_in_window['Effective Precipitation'].sum() if not site_data_in_window.empty else None
            
            # Store the result in the dictionary with the key as the survey date
            results[f'{survey_date}'].append(antecedent_precip)

    # Convert the results dictionary into a DataFrame
    result_df = pd.DataFrame(results)

    # Set the index to be the time windows (1 to 60 days)
    result_df.index = [f'{days}_days' for days in range(1, max_days + 1)]

    return result_df

In [ ]:
Antecedent_EP_LCRW_1 = calculate_antecedent_EP(survey_dates, LCRW_EP_data)
Antecedent_EP_LCRW_1.head()

In [ ]:
from scipy.stats import linregress

def calculate_correlations(ADNL_df, hydro_param_df, hydro_param_name, plot=False):
    correlations = {}
    model_params = {}

    # Get the "Active Network Drainage Length" values as the dependent variable
    y_cor = ADNL_df["Active Network Drainage Length"].values

    # Loop over each time window (1_day, 2_days, etc.) in hydro_param_df
    for days, row in hydro_param_df.iterrows():
        correlations[days] = {}  # Initialize an empty dictionary for each time window

        # Extract the antecedent precipitation for this time window and remove NaNs
        filtered = row.astype(float)
        nan_mask = -np.isnan(filtered)  # Mask for NaN values
        x_cor = filtered.values[nan_mask]  # Filter out NaN values in the x data (precipitation)
        
        # Perform linear regression: correlation between antecedent precipitation (x_cor) and Active Network Drainage Length (y_cor)
        ling = linregress(x_cor, y_cor[nan_mask])
        correlations[days] = ling  # Store the result for the current time window
        
        # Store slope and intercept for each time window in the model_params dictionary
        model_params[days] = {'slope': ling.slope, 'intercept': ling.intercept}

    # Print the maximum r-value and corresponding day(s)
    max_rvalue = max([v.rvalue for v in correlations.values()])
    max_rvalue_days = [days for days, v in correlations.items() if v.rvalue == max_rvalue]

    print(f"Maximum r-value: {max_rvalue:.3f} on day(s) {max_rvalue_days}")

    # Store the slope and intercept for the best time window (maximum r-value)
    best_slope = model_params[max_rvalue_days[0]]['slope']
    best_intercept = model_params[max_rvalue_days[0]]['intercept']
    best_window = max_rvalue_days[0]

    print(f"Best model parameters for {max_rvalue_days[0]} day(s):")
    print(f"Slope: {best_slope:.3f}, Intercept: {best_intercept:.3f}")


    # Plot the correlations if requested, but only once after processing all time windows
    if plot:
        fig, ax1 = plt.subplots()

        # Extract r-values and p-values
        r_values = [v.rvalue for v in correlations.values()]
        p_values = [v.pvalue for v in correlations.values()]

        # Plot r-values (correlation coefficients) on the left y-axis
        ax1.plot(range(1, len(correlations) + 1), r_values, color='tab:blue', label='r-value')
        ax1.set_xlabel("Window length [days]")
        ax1.set_ylabel("r-value", color='tab:blue')
        ax1.tick_params(axis='y', labelcolor='tab:blue')
        ax1.set_ylim(-1, 1)

        # Highlight the max r-value with a vertical line and annotation
        max_rvalue_day = r_values.index(max_rvalue) + 1  # +1 because days are 1-indexed
        ax1.axvline(x=max_rvalue_day, color='tab:red', linestyle='--', linewidth=2, label='Max r-value')

        ax1.annotate(f'Max r = {max_rvalue:.3f} on Day {max_rvalue_day}', 
                     xy=(max_rvalue_day, max_rvalue), 
                     xytext=(max_rvalue_day + 2, max_rvalue),  # Shift annotation to the right
                     arrowprops=dict(arrowstyle="->", lw=1.5),
                     fontsize=10, color='tab:blue')

        # Create a second y-axis to plot p-values
        ax2 = ax1.twinx()
        ax2.plot(range(1, len(correlations) + 1), p_values, color='tab:red', label='p-value')
        ax2.set_ylabel("p-value", color='tab:red')
        ax2.tick_params(axis='y', labelcolor='tab:red')

        # Annotate the p-value at the maximum r-value day
        pvalue_at_max_r = p_values[max_rvalue_day - 1]  # Get p-value at the max r-value day
        ax2.annotate(f'p-value = {pvalue_at_max_r:.3f} on Day {max_rvalue_day}', 
                     xy=(max_rvalue_day, pvalue_at_max_r),  
                     xytext=(max_rvalue_day + 2, pvalue_at_max_r - 0.025),  # Shift annotation to the right
                     arrowprops=dict(arrowstyle="->", lw=1.5),
                     fontsize=10, color='tab:red')

        # Add a title and tight layout for better readability
        ax1.set_title(f"Correlation for {hydro_param_name}")
        plt.tight_layout()
        plt.show()
        

    return correlations, best_window, best_slope, best_intercept

In [ ]:
ADNL_EP1_correlations, ADNL_EP1_best_window, ADNL_EP1_best_slope, ADNL_EP1_best_intercept = calculate_correlations(ADNL_LCRW, Antecedent_EP_LCRW_1, "Antecedent Cumulative Effective Precipitation",True)
# ADNL_EP1_correlations

In [ ]:
ADNL_Precip_correlations, ADNL_Precip_best_window, ADNL_Precip_best_slope, ADNL_Precip_best_intercept = calculate_correlations(ADNL_LCRW, Antecedent_Precip_LCRW, "Antecedent Cumulative Precipitation", True)
# ADNL_Precip_correlations

In [ ]:
ADNL_ET_correlations, ADNL_ET_best_window, ADNL_ET_best_slope, ADNL_ET_best_intercept = calculate_correlations(ADNL_LCRW, Antecedent_ET_LCRW, "Antecedent Cumulative Evapotranspiration" ,True)

In [ ]:
import pickle

# Save all correlations in a pickle file
ADNL_Precip_correlations = calculate_correlations(ADNL_LCRW, Antecedent_Precip_LCRW, "Antecedent Cumulative Precipitation")
with open("ADNL_Precip_correlations_LCRW.pkl", "wb") as f:
    pickle.dump(ADNL_Precip_correlations, f)

# Save all correlations in a pickle file
ADNL_ET_correlations = calculate_correlations(ADNL_LCRW, Antecedent_ET_LCRW, "Antecedent Cumulative Evapotranspiration")
with open("ADNL_ET_correlations_LCRW.pkl", "wb") as f:
    pickle.dump(ADNL_ET_correlations, f)

# Save all correlations in a pickle file
ADNL_EP1_correlations = calculate_correlations(ADNL_LCRW, Antecedent_EP_LCRW_1, "Antecedent Cumulative Effective Precipitation")
with open("ADNL_EP_correlations_LCRW.pkl", "wb") as f:
    pickle.dump(ADNL_EP1_correlations, f)


In [ ]:
# Load the correlations from all the pickle files
with open("ADNL_Precip_correlations_LCRW.pkl", "rb") as f:
    ADNL_Precip_correlations_LCRW = pickle.load(f)
# Load the correlations from the pickle file
with open("ADNL_ET_correlations_LCRW.pkl", "rb") as f:
    ADNL_ET_correlations_LCRW = pickle.load(f)
# Load the correlations from the pickle file
with open("ADNL_EP_correlations_LCRW.pkl", "rb") as f:
    ADNL_EP1_correlations_LCRW = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt

# Create a dictionary for each dataset and its corresponding correlation data for all 4 types
datasets = {
    'LCRW': {
        'Precip': ADNL_Precip_correlations_LCRW,
        'ET': ADNL_ET_correlations_LCRW,
        'EP1': ADNL_EP1_correlations_LCRW
    },
}

def get_rvalues(dataset_name, correlation_type):
    # Assuming it's a tuple and the first element contains the dictionary
    correlation_data = datasets[dataset_name][correlation_type][0] 
    return [correlation_data[f'{day}_days'].rvalue for day in range(1, 60)]

# Define the dataset you're using
dataset_name = 'LCRW'

correlation_types = ['Precip', 'EP1']
label_names = {'Precip': 'Precipitation', 'EP1': 'Effective Precipitation'}
colors_by_type = {'Precip': 'royalblue', 'ET': 'orange', 'EP1': 'tomato'}
time_windows = range(1, 61)  # 1–60 days

plt.figure(figsize=(7, 4))

for corr_type in correlation_types:
    rvalues = get_rvalues(dataset_name, corr_type)

    # Truncate if needed to avoid length mismatch
    min_len = min(len(time_windows), len(rvalues))
    rvalues = rvalues[:min_len]
    time_range = list(time_windows)[:min_len]

    plt.plot(time_range, rvalues, label=label_names[corr_type],
             linestyle='-',
             color=colors_by_type[corr_type])

    # Annotate max point
    max_r = max(rvalues)
    max_day = rvalues.index(max_r) + 1
    plt.annotate(f'Max r = {max_r:.3f} at {max_day} days',
                 xy=(max_day, max_r),
                 xytext=(max_day + 3, max_r - 0.02),
                 arrowprops=dict(arrowstyle="->"),
                 fontsize=9,
                 color=colors_by_type[corr_type])

plt.xlim(0, 60)
plt.ylim(0, 1.0)
plt.xlabel('Antecedent Window Length (Days)')
plt.ylabel('Pearson Correlation (r)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Updated to retrieve both r-values and p-values
def get_r_and_p_values(dataset_name, correlation_type):
    correlation_data = datasets[dataset_name][correlation_type][0]
    r_values = [correlation_data[f'{day}_days'].rvalue for day in range(1, 60)]
    p_values = [correlation_data[f'{day}_days'].pvalue for day in range(1, 60)]
    return r_values, p_values


correlation_types = ['Precip', 'EP1']
label_names = {'Precip': 'Precipitation', 'EP1': 'Effective Precipitation'}
colors_by_type = {'Precip': 'royalblue', 'ET': 'orange', 'EP1': 'tomato'}
time_windows = range(1, 61)  # 1–60 days

fig, ax1 = plt.subplots(figsize=(7, 5))

for corr_type in correlation_types:
    r_values, p_values = get_r_and_p_values('LCRW', corr_type)

    min_len = min(len(time_windows), len(r_values))
    time_range = list(time_windows)[:min_len]
    r_values = r_values[:min_len]
    p_values = p_values[:min_len]

    ax1.plot(time_range, r_values, label=label_names[corr_type],
             linestyle='-', color=colors_by_type[corr_type])

    # Annotate the max r-value
    max_r = max(r_values)
    max_day = r_values.index(max_r) + 1
    ax1.annotate(f'Max r = {max_r:.3f} at {max_day} days',
                 xy=(max_day, max_r),
                 xytext=(max_day + 3, max_r - 0.02),
                 arrowprops=dict(arrowstyle="->"),
                 fontsize=9,
                 color=colors_by_type[corr_type])

# Set axis limits and labels for r-values
ax1.set_xlim(0, 60)
ax1.set_ylim(0, 1.0)
ax1.set_xlabel('Antecedent Window Length (Days)')
ax1.set_ylabel('Pearson Correlation (r)')
ax1.grid(True)
ax1.legend(loc='upper left')

# Create secondary y-axis for p-values
ax2 = ax1.twinx()

for corr_type in correlation_types:
    _, p_values = get_r_and_p_values('LCRW', corr_type)
    p_values = p_values[:60]
    ax2.plot(time_range, p_values, linestyle='--',
             color=colors_by_type[corr_type], alpha=0.6)

# Add horizontal dotted line for p = 0.001
ax2.axhline(y=0.01, color='gray', linestyle=':', linewidth=1.2, label='p = 0.01 (significance)')

ax2.set_ylabel("p-value", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Optional: add a combined legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels , loc='upper right', ncol=1)

ax2.text(61, 0.01, 'p = 0.01', 
         va='center', ha='left', fontsize=10, color='gray')

      

plt.title('Correlation and p-values vs. Antecedent Window Length')
plt.tight_layout()
plt.show()


# Predictive Model

In [ ]:
def calculate_antecedent_EP_best_window(start_date, end_date, site_data, best_window):
    """
    Calculate antecedent precipitation for each day between start_date and end_date using the best time window.
    Ensures consistency with the other function's method (with 6-day window and inclusive date range).

    Parameters:
    - start_date: The start date of the prediction period.
    - end_date: The end date of the prediction period.
    - site_data: DataFrame containing precipitation data with 'datetime' and 'Effective Precipitation' columns.
    - best_window: The best time window for antecedent precipitation (in days, format '6_days').

    Returns:
    - result_df: DataFrame with antecedent precipitation for each survey date and the best window length.
    """

    # Make a copy of site_data to avoid modifying the original DataFrame
    site_data_copy = site_data.copy()

    # Ensure 'datetime' is in the correct format and set it as the index
    site_data_copy['datetime'] = pd.to_datetime(site_data_copy['datetime'])

    # If datetime is timezone-aware, make it timezone-naive
    if site_data_copy['datetime'].dt.tz is not None:
        site_data_copy['datetime'] = site_data_copy['datetime'].dt.tz_localize(None)
    
    site_data_copy.set_index('datetime', inplace=True)
    
    # Convert start_date and end_date to datetime, and make them timezone-naive if they are not already
    start_date = pd.to_datetime(start_date).normalize()  # Removing time component for comparison
    end_date = pd.to_datetime(end_date).normalize()

    # Extract the numerical value for the best window (e.g., '6_days' -> 6)
    best_window_days = int(best_window.split('_')[0])

    # Generate the list of survey dates (from start_date to end_date)
    survey_dates = pd.date_range(start=start_date, end=end_date)
    
    # Initialize a list to hold the results
    results = []

    # Loop through each survey date
    for survey_date in survey_dates:
        # Ensure survey_date is a datetime object
        survey_date = pd.to_datetime(survey_date)

        # Calculate the start date for the antecedent precipitation window (inclusive of start_date_window)
        start_date_window = survey_date - pd.Timedelta(days=best_window_days)

        # Filter data for the current site within the best time window (inclusive start_date_window to survey_date)
        site_data_in_window = site_data_copy.loc[(site_data_copy.index >= start_date_window) & (site_data_copy.index <= survey_date)]

        # Calculate the antecedent precipitation by summing the precipitation values in the window
        antecedent_precip = site_data_in_window['Effective Precipitation'].sum() if not site_data_in_window.empty else None

        # Append the result to the list
        results.append([survey_date, antecedent_precip])

    # Convert the results into a DataFrame
    result_df = pd.DataFrame(results, columns=['Survey_Date', 'Antecedent_EffectivePrecip'])

    # Set the index to be the survey dates for better readability
    result_df.set_index('Survey_Date', inplace=True)

    return result_df


In [ ]:
Antecedent_EP_bestwindow =  calculate_antecedent_EP_best_window('05-01-2024', '06-29-2025', LCRW_EP_data, ADNL_EP1_best_window)
Antecedent_EP_dry = Antecedent_EP_bestwindow.copy()
# Reset Pandas display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
# Antecedent_EP_bestwindow

In [ ]:
L_max = 71.28007300000002

Antecedent_EP_bestwindow['Predicted ADNL_Linear'] = Antecedent_EP_bestwindow['Antecedent_EffectivePrecip']*ADNL_EP1_best_slope + ADNL_EP1_best_intercept
Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped'] = np.minimum(Antecedent_EP_bestwindow['Antecedent_EffectivePrecip']*ADNL_EP1_best_slope + ADNL_EP1_best_intercept, L_max)
print(ADNL_EP1_best_slope, ADNL_EP1_best_intercept)
Antecedent_EP_bestwindow

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from scipy.stats import pearsonr

# Set Survey Date as the index to align on date
actual_series = ADNL_LCRW_copy.set_index('Survey Date')['Active Network Drainage Length']
predicted_series_EP = Antecedent_EP_bestwindow['Predicted ADNL_Linear']

# Convert index with underscores to datetime by first replacing '_' with '-'
actual_series.index = pd.to_datetime(actual_series.index.str.replace('_', '-'))

# Align both series on the index (i.e., matching dates)
actual_aligned, predicted_aligned = actual_series.align(predicted_series_EP, join='inner')

# Calculate metrics
mae = mean_absolute_error(actual_aligned, predicted_aligned)
mse = mean_squared_error(actual_aligned, predicted_aligned)
rmse = np.sqrt(mse)
r2 = r2_score(actual_aligned, predicted_aligned)

# Calculate Pearson correlation coefficient (R)
r_value, p_value = pearsonr(actual_aligned, predicted_aligned)

# Output the results
print(f"----For the EP 6 Linear Model----")
print(f"MAE: {mae:.3f}")
print(f"MSE: {mse:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R² Score: {r2:.3f}")
print(f"R (Pearson correlation): {r_value:.3f}")
print(f"p-value: {p_value:.10f}")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from scipy.stats import pearsonr

# Set Survey Date as the index to align on date
actual_series = ADNL_LCRW_copy.set_index('Survey Date')['Active Network Drainage Length']
predicted_series_EP_capped = Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped']

# Convert index with underscores to datetime by first replacing '_' with '-'
actual_series.index = pd.to_datetime(actual_series.index.str.replace('_', '-'))

# Align both series on the index (i.e., matching dates)
actual_aligned, predicted_aligned_capped = actual_series.align(predicted_series_EP_capped, join='inner')

# Calculate metrics
mae = mean_absolute_error(actual_aligned, predicted_aligned_capped)
mse = mean_squared_error(actual_aligned, predicted_aligned_capped)
rmse = np.sqrt(mse)
r2 = r2_score(actual_aligned, predicted_aligned_capped)

# Calculate Pearson correlation coefficient (R)
r_value, p_value = pearsonr(actual_aligned, predicted_aligned_capped)

# Output the results
print(f"----For the EP 6 Capped Linear Model----")
print(f"MAE: {mae:.3f}")
print(f"MSE: {mse:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R² Score: {r2:.3f}")
print(f"R (Pearson correlation): {r_value:.3f}")
print(f"p-value: {p_value:.10f}")

In [ ]:
EP_df = Antecedent_EP_bestwindow['Antecedent_EffectivePrecip']

# Combine into a DataFrame based on index
df = pd.concat([
    actual_aligned.rename('Actual'),
    predicted_aligned.rename('Predicted'),
    predicted_aligned_capped.rename('PredictedCapped'),
    Antecedent_EP_bestwindow['Antecedent_EffectivePrecip'].rename('Antecedent_EffectivePrecip')
], axis=1).dropna()


In [ ]:
# Ensure 'Survey Date' and 'Predicted_ADNL['date']' are naive datetime objects
ADNL_LCRW_copy['Survey Date'] = pd.to_datetime(ADNL_LCRW_copy['Survey Date'], format='%Y_%m_%d').dt.tz_localize(None)

# # Remove timezone info from the index of Antecedent_Precip_bestwindow
# Antecedent_Precip_bestwindow.index = Antecedent_Precip_bestwindow.index.tz_localize(None)

# Remove timezone info from the index of Antecedent_EP_bestwindow
Antecedent_EP_bestwindow.index = Antecedent_EP_bestwindow.index.tz_localize(None)

# Create figure and primary axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot Predicted ADNL (EP) on primary y-axis
line1, = ax1.plot(
    Antecedent_EP_bestwindow.index,
    Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped'],
    color='orchid', linestyle='--', label='Predicted ADNL (EP)', zorder=2
)

# Plot Observed ADNL on same primary y-axis
line2, = ax1.plot(
    ADNL_LCRW_copy['Survey Date'],
    ADNL_LCRW_copy['Active Network Drainage Length'],
    marker='o', linestyle='', color='blue', label='Observed ADNL', zorder=3
)

# Set label for the y-axis
ax1.set_ylabel('Active Network Drainage Length (km)', color='black')

# Add third axis for Precipitation
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 10))  # Offset to avoid overlap
bar3 = ax3.bar(
    gridded_precip_LCRW.index,
    -gridded_precip_LCRW['LCRW'],
    width=1, align='center', alpha=0.5, color='black',
    label='Precipitation', zorder=1
)
ax3.set_ylabel('Precipitation (mm)', color='black')

# Set y-axis limits
ax1.set_ylim(50, 85)     # For ADNL
ax3.set_ylim(-100, 0)    # For precipitation

# Set x-axis limits
start_date = pd.to_datetime('2024-05-01')
end_date = pd.to_datetime('2024-11-30')
ax1.set_xlim(start_date, end_date)
ax3.set_xlim(start_date, end_date)

# Add gridlines
ax1.grid(True, which='both', linestyle='--', color='gray', alpha=0.5)

# Combine all legend handles/labels
handles1, labels1 = ax1.get_legend_handles_labels()
handles3, labels3 = ax3.get_legend_handles_labels()

# Add unified legend below the plot
fig.legend(
    handles1 + handles3,
    labels1 + labels3,
    loc='upper center',
    bbox_to_anchor=(0.5, 0.1),
    ncol=3,
    frameon=True
)

# Adjust layout to make room for legend
plt.tight_layout(rect=[0, 0.08, 1, 1])

# Show plot
plt.show()

In [ ]:
# Merge on the index (date)
Antecedent_EP_dry = Antecedent_EP_dry.merge(dry_days_df, left_index=True, right_index=True, how='left')
Antecedent_EP_dry.reset_index(inplace=True)
Antecedent_EP_dry
# Rename 'Survey Date' in ADNL to match 'Survey_Date' in Antecedent_EP_dry
ADNL_LCRW_copy.rename(columns={'Survey Date': 'Survey_Date'}, inplace=True)

# Now merge on 'Survey_Date'
Antecedent_EP_dry = Antecedent_EP_dry.merge(
    ADNL_LCRW_copy[['Survey_Date', 'Active Network Drainage Length']],
    on='Survey_Date',
    how='left'
)

Antecedent_EP_dry


In [ ]:

# Drop rows with missing ADNL values
Antecedent_EP_dry_clean = Antecedent_EP_dry.dropna(subset=['Active Network Drainage Length'])
Antecedent_EP_dry_clean.to_pickle(r'C:\Users\mjg9941\Documents\PhD_Research\Projects\SmartWater\Python_Projects\Field_Campaigns\dataretrieval\Antecedent_EP_dry_clean.pkl')
Antecedent_EP_dry_clean

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Assume df is your DataFrame with no missing values
df_dry = Antecedent_EP_dry_clean

# Rename for clarity
df_dry = df_dry.rename(columns={
    'Antecedent_EffectivePrecip': 'EP',
    'Dry_Days_Before_LCRW': 'DryDays',
    'Active Network Drainage Length': 'ADNL'
})

# Construct custom design matrix: [EP, -DryDays]
X = df_dry[['EP', 'DryDays']].copy()
X['DryDays'] = -X['DryDays']  # Enforce subtraction

y = df_dry['ADNL']

# Fit the model
model = LinearRegression()
model.fit(X, y)

# Get model coefficients
x1, x2 = model.coef_
b = model.intercept_

# Predict and apply cap
y_pred_raw = model.predict(X)
y_pred_capped = np.minimum(y_pred_raw, L_max)

# Print fitted equation
print(f"Fitted model: ADNL = ({x1:.4f} * EP) + ({x2:.4f} * DryDays) + {b:.4f}")
print(f"Capping predictions at ADNL_max = {L_max:.4f}")

# Evaluate model performance using capped predictions
r2 = r2_score(y, y_pred_capped)
rmse = np.sqrt(mean_squared_error(y, y_pred_capped))
mae = mean_absolute_error(y, y_pred_capped)

# Step 4: Calculate Pearson correlation coefficient (R)
r_value, p_value = pearsonr(y, y_pred_capped)

print(f"\nModel Performance (Capped):")
print(f"----For the EP 6 Multiparameter Model----")
print(f"R²:   {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R:    {r_value:.4f}")
print(f"p:    {p_value:.10f}")


In [ ]:
import matplotlib.ticker as ticker
# Apply formula and cap each predicted value at L_max
Antecedent_EP_dry['Predicted_ADNL'] = np.minimum(
    (x1 * Antecedent_EP_dry['Antecedent_EffectivePrecip']) +
    (x2 * Antecedent_EP_dry['Dry_Days_Before_LCRW']) +
    b,
    L_max
)

# Set index to Survey_Date
Antecedent_EP_dry.index = Antecedent_EP_dry['Survey_Date']

Antecedent_EP_dry


In [ ]:
ADNL_LCRW_copy

In [ ]:
# Fix Survey Date parsing (no need for format if it's already YYYY-MM-DD)
ADNL_LCRW_copy['Survey_Date'] = pd.to_datetime(ADNL_LCRW_copy['Survey_Date']).dt.tz_localize(None)

# Ensure predicted series indices are datetime and tz-naive
Antecedent_EP_dry.index = pd.to_datetime(Antecedent_EP_dry.index).tz_localize(None)
Antecedent_EP_bestwindow.index = pd.to_datetime(Antecedent_EP_bestwindow.index).tz_localize(None)
gridded_precip_LCRW.index = pd.to_datetime(gridded_precip_LCRW.index).tz_localize(None)

# Plot
fig, ax1 = plt.subplots(figsize=(8, 6))

# Observed ADNL
line_obs, = ax1.plot(
    ADNL_LCRW_copy['Survey_Date'], ADNL_LCRW_copy['Active Network Drainage Length'],
    marker='o', linestyle='', color='blue', label='Observed ADNL', zorder=3
)

# Predicted (Multilinear)
line_multi, = ax1.plot(
    Antecedent_EP_dry.index, Antecedent_EP_dry['Predicted_ADNL'],
    color='tomato', linestyle='-', label='Multilinear Model', zorder=2
)

# Predicted (Linear - capped)
line_lin_cap, = ax1.plot(
    Antecedent_EP_bestwindow.index, Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped'],
    color='orchid', linestyle='--', label='Linear Model (Capped)', zorder=2
)

# Predicted (Linear - uncapped)
line_lin_raw, = ax1.plot(
    Antecedent_EP_bestwindow.index, Antecedent_EP_bestwindow['Predicted ADNL_Linear'],
    color='mediumseagreen', linestyle='-.', label='Linear Model (Raw)', zorder=2
)

# Y-axis ADNL
ax1.set_ylabel('Active Network Drainage Length (km)', color='black')
ax1.set_ylim(50, 85)

# Add precipitation on secondary axis
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 10))
bar_precip = ax3.bar(
    gridded_precip_LCRW.index,
    -np.clip(gridded_precip_LCRW['LCRW'], None, 45),
    width=1, align='center', alpha=0.5, color='black',
    label='Precipitation', zorder=1
)
ax3.set_ylabel('Precipitation (mm)', color='black')
ax3.set_ylim(-100, 0)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{-int(x)}"))

# Date range
start_date = pd.to_datetime('2024-05-01')
end_date = pd.to_datetime('2024-12-01')
ax1.set_xlim(start_date, end_date)
ax3.set_xlim(start_date, end_date)

# Grid and layout
ax1.grid(True, which='both', linestyle='-', color='gray', alpha=0.5)
plt.title("Observed vs Predicted ADNL with Precipitation")
plt.tight_layout(rect=[0, 0, 1, 0.95])

# # Combined legend
# handles, labels = ax1.get_legend_handles_labels()
# handles2, labels2 = ax3.get_legend_handles_labels()
# ax1.legend(handles + handles2, labels + labels2, loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

# Show
plt.show()


In [ ]:
# Fix Survey Date parsing (no need for format if it's already YYYY-MM-DD)
ADNL_LCRW_copy['Survey_Date'] = pd.to_datetime(ADNL_LCRW_copy['Survey_Date']).dt.tz_localize(None)

# Ensure predicted series indices are datetime and tz-naive
Antecedent_EP_dry.index = pd.to_datetime(Antecedent_EP_dry.index).tz_localize(None)
Antecedent_EP_bestwindow.index = pd.to_datetime(Antecedent_EP_bestwindow.index).tz_localize(None)
gridded_precip_LCRW.index = pd.to_datetime(gridded_precip_LCRW.index).tz_localize(None)

# Plot
fig, ax1 = plt.subplots(figsize=(8, 6))

# Observed ADNL
line_obs, = ax1.plot(
    ADNL_LCRW_copy['Survey_Date'], ADNL_LCRW_copy['Active Network Drainage Length'],
    marker='o', linestyle='', color='blue', label='Observed ADNL', zorder=3
)

# Predicted (Multilinear)
line_multi, = ax1.plot(
    Antecedent_EP_dry.index, Antecedent_EP_dry['Predicted_ADNL'],
    color='tomato', linestyle='-', label='Multilinear Model', zorder=2
)

# # Predicted (Linear - capped)
# line_lin_cap, = ax1.plot(
#     Antecedent_EP_bestwindow.index, Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped'],
#     color='orchid', linestyle='--', label='Linear Model (Capped)', zorder=2
# )

# # Predicted (Linear - uncapped)
# line_lin_raw, = ax1.plot(
#     Antecedent_EP_bestwindow.index, Antecedent_EP_bestwindow['Predicted ADNL_Linear'],
#     color='mediumseagreen', linestyle='-.', label='Linear Model (Raw)', zorder=2
# )

# Y-axis ADNL
ax1.set_ylabel('Active Network Drainage Length (km)', color='black')
ax1.set_ylim(50, 85)

# Add precipitation on secondary axis
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 10))
bar_precip = ax3.bar(
    gridded_precip_LCRW.index,
    -np.clip(gridded_precip_LCRW['LCRW'], None, 45),
    width=1, align='center', alpha=0.5, color='black',
    label='Precipitation', zorder=1
)
ax3.set_ylabel('Precipitation (mm)', color='black')
ax3.set_ylim(-100, 0)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{-int(x)}"))

# Date range
start_date = pd.to_datetime('2024-05-01')
end_date = pd.to_datetime('2024-12-01')
ax1.set_xlim(start_date, end_date)
ax3.set_xlim(start_date, end_date)

# Grid and layout
ax1.grid(True, which='both', linestyle='-', color='gray', alpha=0.5)
# plt.title("Observed vs Predicted ADNL with Precipitation")
plt.tight_layout(rect=[0, 0, 1, 0.95])

# # Combined legend
# handles, labels = ax1.get_legend_handles_labels()
# handles2, labels2 = ax3.get_legend_handles_labels()
# ax1.legend(handles + handles2, labels + labels2, loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

# Show
plt.show()


In [ ]:
# Example: reindex all to match observed dates
observed = ADNL_all.set_index(pd.to_datetime(ADNL_all['Survey Date'], format='%Y_%m_%d'))['Active Network Drainage Length']
ep_multilinear = Antecedent_EP_dry['Predicted_ADNL'].reindex(observed.index).dropna()
ep_linear = Antecedent_EP_bestwindow['Predicted ADNL_Linear'].reindex(observed.index).dropna()
ep_capped_linear = Antecedent_EP_bestwindow['Predicted ADNL_Linear_capped'].reindex(observed.index).dropna()

# Align all three with observed for shared dates
shared_index = observed.index.intersection(ep_multilinear.index).intersection(ep_linear.index)
observed = observed.loc[shared_index]
ep_multilinear = ep_multilinear.loc[shared_index]
ep_linear = ep_linear.loc[shared_index]
ep_capped_linear = ep_capped_linear.loc[shared_index]

# Filter for ADNL < 55
mask_below_55 = observed < 55
observed_below_55 = observed[mask_below_55]
ep_multilinear_below_55 = ep_multilinear[mask_below_55]
ep_linear_below_55 = ep_linear[mask_below_55]
ep_capped_linear_below_55 = ep_capped_linear[mask_below_55]

# RMSE calculation
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Print RMSEs
print("RMSE below 55 km:")
print("Multilinear (EP):", rmse(observed_below_55, ep_multilinear_below_55))
print("Linear (EP):", rmse(observed_below_55, ep_linear_below_55))
print("Capped (EP):", rmse(observed_below_55, ep_capped_linear_below_55))
